In [1]:
import torch

In [ ]:
torch.nn.Module

In [9]:
torch.ones(1)/(torch.zeros(1))

tensor([inf])

In [1]:
!unzip -o "/storage/vskovoroda/Stones/full_dataset.zip" -d "/storage/vskovoroda/Stones/"

Archive:  /storage/vskovoroda/Stones/full_dataset.zip
  inflating: /storage/vskovoroda/Stones/Asbest veins in the open pit conditions/parts45769.zip  
  inflating: /storage/vskovoroda/Stones/Asbest veins in the open pit conditions/part2.zip  
  inflating: /storage/vskovoroda/Stones/Asbest veins in the open pit conditions/part1.zip  
  inflating: /storage/vskovoroda/Stones/Asbest veins in the open pit conditions/part11.zip  
  inflating: /storage/vskovoroda/Stones/Asbest veins in the open pit conditions/part3.zip  


In [2]:
!mv -f "/storage/vskovoroda/Stones/Asbest veins in the open pit conditions/parts45769.zip" -t "/storage/vskovoroda/Stones/"
!mv -f "/storage/vskovoroda/Stones/Asbest veins in the open pit conditions/part2.zip" -t "/storage/vskovoroda/Stones/"
!mv -f "/storage/vskovoroda/Stones/Asbest veins in the open pit conditions/part1.zip" -t "/storage/vskovoroda/Stones/"
!mv -f "/storage/vskovoroda/Stones/Asbest veins in the open pit conditions/part11.zip" -t "/storage/vskovoroda/Stones/"
!mv -f "/storage/vskovoroda/Stones/Asbest veins in the open pit conditions/part3.zip" -t "/storage/vskovoroda/Stones/"

In [3]:
!rm -d "/storage/vskovoroda/Stones/Asbest veins in the open pit conditions"
!mkdir "/storage/vskovoroda/Stones/data/"
!mkdir "/storage/vskovoroda/Stones/data/part1"\
        "/storage/vskovoroda/Stones/data/part2"\
        "/storage/vskovoroda/Stones/data/part3"\
        "/storage/vskovoroda/Stones/data/part4"\
        "/storage/vskovoroda/Stones/data/part5"

mkdir: cannot create directory ‘/storage/vskovoroda/Stones/data/’: File exists
mkdir: cannot create directory ‘/storage/vskovoroda/Stones/data/part1’: File exists
mkdir: cannot create directory ‘/storage/vskovoroda/Stones/data/part2’: File exists
mkdir: cannot create directory ‘/storage/vskovoroda/Stones/data/part3’: File exists
mkdir: cannot create directory ‘/storage/vskovoroda/Stones/data/part4’: File exists
mkdir: cannot create directory ‘/storage/vskovoroda/Stones/data/part5’: File exists


In [21]:
!unzip -qq -o "/storage/vskovoroda/Stones/part1.zip" -d "/storage/vskovoroda/Stones/data/part1"
!unzip -qq -o "/storage/vskovoroda/Stones/part2.zip" -d "/storage/vskovoroda/Stones/data/part2"
!unzip -qq -o "/storage/vskovoroda/Stones/part3.zip" -d "/storage/vskovoroda/Stones/data/part3"
!unzip -qq -o "/storage/vskovoroda/Stones/part11.zip" -d "/storage/vskovoroda/Stones/data/part4"
!unzip -qq -o "/storage/vskovoroda/Stones/parts45769.zip" -d "/storage/vskovoroda/Stones/data/part5"

In [4]:
!rm "/storage/vskovoroda/Stones/part1.zip"
!rm "/storage/vskovoroda/Stones/part2.zip"
!rm "/storage/vskovoroda/Stones/part3.zip"
!rm "/storage/vskovoroda/Stones/part11.zip"
!rm "/storage/vskovoroda/Stones/parts45769.zip"

## Подготовка изображений

In [1]:
from pycocotools.coco import COCO
import cv2
import os
import json
import numpy as np
import multiprocessing

In [2]:
def make_mask(image):
    global coco, part
    # if not os.path.isfile(part + '/images/' + image['file_name']):
    #     return
    ann_ids = coco.getAnnIds(
                    imgIds = image['id'], 
                    catIds = [1],  # id асбеста
                    iscrowd=None
                )
    anns = coco.loadAnns(ann_ids)
    mask = np.zeros((image['height'],image['width']))
    for ann in anns:
        mask = np.maximum(mask,coco.annToMask(ann))
    
    img = cv2.imread(part + '/images/' + image['file_name'])
    # Сохраняем изображение
    os.system('cp ' + part + '/images/' + image['file_name'] + ' images/' + image['file_name'][:-4] + '.bmp')
    
    
    img_t = cv2.blur(img, (32, 32))
    img[mask<1] = img_t[mask<1]
    
    # сохраняем blur изображение
    cv2.imwrite(start_dir + '/blur_images/' + image['file_name'][:-4] + '.bmp', img)
    
    cv2.imwrite(start_dir + '/annotations/' + image['file_name'][:-4]+'.bmp', mask) # сохраняем маску
    
    # os.system('cp ' + part + '/images/' + image['file_name'] + ' images/' + image['file_name'][:-4] + '.bmp')
    # os.replace(part + '/images/' + image['file_name'], 'images/' + image['file_name'][:-4] + '.bmp')

In [18]:
!rm -dr "/storage/vskovoroda/Stones/data/images"
!rm -dr "/storage/vskovoroda/Stones/data/blur_images"
!rm -dr "/storage/vskovoroda/Stones/data/annotations"

In [19]:
if not os.path.exists('images'):
    os.mkdir('images')
if not os.path.exists('blur_images'):
    os.mkdir('blur_images')
if not os.path.exists('annotations'):
    os.mkdir('annotations')

for part in ['part1', 'part2', 'part3', 'part4', 'part5']:
    print(part, 'Start')
    ann_file = start_dir + part + '/annotations/instances_default.json'
    coco = COCO(ann_file)
    pool_obj = multiprocessing.Pool()
    pool_obj.map(make_mask, list(coco.imgs.values()))
    print(part, 'Done!')

part1 Start
loading annotations into memory...
Done (t=0.34s)
creating index...
index created!
part1 Done!
part2 Start
loading annotations into memory...
Done (t=0.55s)
creating index...
index created!
part2 Done!
part3 Start
loading annotations into memory...
Done (t=0.92s)
creating index...
index created!
part3 Done!
part4 Start
loading annotations into memory...
Done (t=0.82s)
creating index...
index created!
part4 Done!
part5 Start
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
part5 Done!
